<a href="https://colab.research.google.com/github/uob-positron-imaging-centre/PEPT-Algorithms-RoPP/blob/main/LineDensity_RoPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank"  href="https://github.com/uob-positron-imaging-centre/pept"><img src="https://github.com/uob-positron-imaging-centre/misc-hosting/blob/master/logo.png?raw=true" style="height:200px; display: block; margin-left: auto; margin-right: auto;"/></a>

# Interactive PEPT Analysis Examples using *the Line Density Algorithm*

> [1] Bickell M, Buffler A, Govender I, Parker DJ. A new line density tracking algorithm for PEPT and its application to multiple tracers. Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment. 2012 Aug 1;682:36-41.

---

#### Copyright 2021 the `pept` developers
##### Jupyter Notebook authored by Andrei Leonard Nicusan and Dr. Kit Windows-Yule for the "PEPT: A Comparative Review" paper, commissioned by the Reports on Progress in Physics journal

Licensed under the GNU License, Version 3.0 (the "License").

---


# 1. Introduction

Positron emission particle tracking (PEPT) is a powerful technique allowing the non-invasive, three-dimensional tracking of one or more radioactive 'tracer' particles through particulate, fluid or multiphase systems. It allows particle or fluid motion to be tracked with sub-millimetre accuracy and sub-millisecond temporal resolution and, due to its use of highly-penetrating 511keV gamma rays, can be used to probe the internal dynamics of even large, dense, optically opaque systems <sup>[[2]](https://www.sciencedirect.com/science/article/pii/016890029390864E) [[3]](https://www.sciencedirect.com/science/article/pii/S0263876208003341) [[4]](https://aip.scitation.org/doi/abs/10.1063/1.4983046@rsi.2017.IMGP2017.issue-1)</sup>. In light of its versatility both in terms of the scales and materials of particles which can be tracked <sup>[[5]](https://www.sciencedirect.com/science/article/pii/S1672251507001455)[[6]](https://www.sciencedirect.com/science/article/pii/S0168900206005341)</sup>, and the sizes and geometries of the systems which can be imaged <sup>[[7]](https://www.sciencedirect.com/science/article/pii/S0168900209001880) [[8]](https://www.sciencedirect.com/science/article/pii/S0029549316000273)</sup> , the technique has wide-ranging applicability in diverse scientific, industrial and biomedical applications.

PEPT is performed by radioactively labelling a particle with a positron-emitting radioisotope such as Fluorine-18 ($^{18}\mathrm{F}$) or Gallium-68 ($^{68}\mathrm{Ga}$), and using the back-to-back gamma rays produced by electron-positron annihilation events in and around the tracer to triangulate its spatial position. Each detected gamma ray represents a **line of response (LoR)** .

## 1.1. This Jupyter Notebook

This interactive Jupyter Notebook illustrates the main processing steps employed by the Line Density algorithm<sup>[1]</sup> for radioactive tracer tracking, as described in the Reports on Progress in Physics "PEPT: A Comparative Review" paper.

An [example dataset](https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv) is used from an experiment run at the University of Birmingham Positron Imaging Centre using the ADAC Forté by Matthew Herald. It consists of a single 1 mm diameter MCC particle activated with Fluorine-18 radioactive tracer material inside a bubbling fluidised bed. The fluidised bed was filled with 90% sand and 10% MCC; air was fed into the bottom of the bed at a rate of 37 litres per minute at 3.5 bar. This dataset was chosen for its high quality captured lines of response, with the tracer still depicting the random particle motion that is inherent to bubbling fluidised beds - and typical in Lagrangian particle tracking.

The [`pept`](https://github.com/uob-positron-imaging-centre/pept) Python library is used for initialising and visualising PEPT data. While not required *per se* for illustrating PEPT algorithms' processing steps, it significantly reduces the amount of repetitive code and visual noise, allowing the reader to focus on the main conceptual procedures.

## 1.2. Running Code Cells
Select any code cell and click on the (▶) sign in the top-left of the cell's frame to run its code (note that code cells have to be run in order when running for the first time).

In [1]:
# First install the `pept` library using pip, Python's package manager
!pip install pept

# 2. The Line Density Method

## 2.1. Read in Line of Response Data
In this initial step, we read in the "raw" LoR data from which our tracer positions are calculated. In this simple example, we load only a single sample (i.e. one particle location at one point in time). The LoRs corresponding to this sample are output as an image.

As discussed in the main text, varying the numbers of LoRs used to calculate a PEPT location can affect the quality of the final location measured. Try altering the value `nrows` below to see how this inluences your results.

In [2]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 50,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.000e+00, 0.000e+00, 1.687e+02, 6.000e+02, 1.428e+02],
       [1.000e-01, 0.000e+00, 1.676e+02, 6.000e+02, 3.139e+02],
       [1.000e-01, 0.000e+00, 4.100e+02, 6.000e+02, 2.401e+02],
       [2.000e-01, 0.000e+00, 2.962e+02, 6.000e+02, 4.525e+02],
       [2.000e-01, 0.000e+00, 1.151e+02, 6.000e+02, 3.534e+02],
       [2.000e-01, 0.000e+00, 1.322e+02, 6.000e+02, 2.661e+02],
       [2.000e-01, 0.000e+00, 3.735e+02, 6.000e+02, 1.310e+02],
       [3.000e-01, 0.000e+00, 1.115e+02, 6.000e+02, 3.534e+02],
       [4.000e-01, 0.000e+00, 2.094e+02, 6.000e+02, 2.808e+02],
       [4.000e-01, 0.000e+00, 2.749e+02, 6.000e+02, 3.062e+02],
       [5.000e-01, 0.000e+00, 2.389e+02, 6.000e+02, 1.681e+02],
       [6.000e-01, 0.000e+00, 4.283e+02, 6.000e+02, 7.020e+01],
       [6.000e-01, 0.000e+00, 1.333e+02, 6.000e+02, 3.499e+02],
       [6.000e-01, 0.000e+00, 3.050e+02, 6.000e+02, 1.032e+02],
       [6.000e-01, 0.000e+00, 9.200e+01, 6.000e+02, 3.929e+02],
       [6.000e-01, 0.000e+00, 1.859e+02,

In [3]:
from pept.visualisation import PlotlyGrapher2D

grapher = PlotlyGrapher2D()
grapher.add_lines(lors)
grapher.show()

## 2.2. Voxelise / Pixelise the Lines of Response
The next step is to 'voxelise' the LoRs - that is, to create a grid of square or rectangular cells, and count the number of lines passing through each. For simplicity and ease of visualisation, we work here in 2D, but the real algorithm of course works in 3D.

In the code below you can set the resolution - i.e. pixel size - by varying the pair of values corresponding to the `resolution` parameter. Typing, for example, (20,20) will give a 20 by 20 grid of pixels.

Try some different values for the `resolution` parameter. As discussed in the main text, larger values - i.e. a higher resolution, smaller pixels - will allow the centroid of our particle to be determined more precisely. However, you may notice that as you increase these values your processor may take longer in the calculation and plotting steps below. Now imagine this is just one of many thousands of locations, and you may get an idea of the issue of using an overly-fine grid. As discussed in the main text, when using codes relying on voxelisation, there is a fine balance to be made between precision and compuationaly efficiency.

In [4]:
resolution = (20, 20)
pixels = pept.Pixels.from_lines(lors, resolution)

pixels

Initialised Pixels class in 0.00043582916259765625 s.


Class instance that inherits from `pept.Pixels`.
Type:
<class 'pept.base.pixel_data.Pixels'>

Attributes
----------
[[ 0.  2.  3.  7.  7.  7.  5.  4.  2. 10.  5.  2.  3.  6.  5.  0.  0.  0.
   0.  0.]
 [ 0.  0.  2.  3.  8.  8.  7.  4.  4.  9.  3.  2.  3.  6.  2.  0.  0.  0.
   0.  0.]
 [ 0.  0.  2.  4.  8. 10.  7.  3.  5.  8.  3.  2.  8.  6.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  3.  4. 14.  8.  4.  7.  8.  5.  3.  7.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  2.  3. 13. 11.  6.  7.  6.  5.  8.  6.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  3.  8. 13.  8.  9.  7.  7.  8.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  2.  5. 11. 13. 10.  6.  8.  5.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  1.  5. 10. 18. 10. 10.  7.  3.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  1.  2.  6. 19. 14. 10.  4.  4.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  2.  2.  5. 19. 23.  8.  3.  2.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  2.  2.  4. 18. 

In [5]:
grapher = PlotlyGrapher2D()

grapher.add_pixels(pixels)
grapher.add_lines(lors, color = "red")

grapher.show()

## 2.3. Fit 1D Gaussians around the Peak

It is interesting to note that, compared to other methods found in these examples, the line-density method actually possesses relatively few parameters.

Having created the line density grid as shown above, the next step is simply to fit a Gaussian function to the datam, as described in the main text - a process which requires no additional user-defined variables!

The means of the fitted Gaussians, corresponding to the estimated particle positions, are then simply returned by the algorithm.

In [6]:
from scipy.optimize import curve_fit

# Find indices of maximum `pixels` value
max_indices = np.unravel_index(pixels.argmax(), pixels.shape)

# Define scaled Gauss model function to fit to data
def gauss(x, mu, sigma, scale):
    return scale / sigma / np.sqrt(2 * np.pi) * np.exp(-0.5 * (x - mu)**2 / sigma**2)

# Fit the Gaussian curve to the pixel values in the horizontal and vertical pixel lines passing
# through the maximum pixel value
pixels_x = np.linspace(pixels.xlim[0] + pixels.pixel_size[0] / 2,
                       pixels.xlim[1] - pixels.pixel_size[0] / 2,
                       pixels.shape[0])
pixels_y = np.linspace(pixels.ylim[0] + pixels.pixel_size[1] / 2,
                       pixels.ylim[1] - pixels.pixel_size[1] / 2,
                       pixels.shape[1])

# Initial guesses
p0_x = np.array([pixels_x[max_indices[0]], pixels.pixel_size[0], 1.])
p0_y = np.array([pixels_y[max_indices[1]], pixels.pixel_size[1], 1.])

opt_x, _ = curve_fit(gauss, pixels_x, pixels[:, max_indices[1]], p0_x)
opt_y, _ = curve_fit(gauss, pixels_y, pixels[max_indices[0], :], p0_y)

# Fitted Gauss curves around the X and Y pixel lines
gauss_xx = np.linspace(pixels.xlim[0], pixels.xlim[1], 1000)
gauss_xy = gauss(gauss_xx, *opt_x)

gauss_yx = np.linspace(pixels.ylim[0], pixels.ylim[1], 1000)
gauss_yy = gauss(gauss_yx, *opt_y)

# The tracer's location is given by the Gaussians' centres
location = np.array([opt_x[0], opt_y[0]])

# Insert the tracer positions' timestamp at column 0
location = np.insert(location, 0, lors[:, 0].mean())

location

array([  0.938     , 314.60901246, 255.28668393])

In [7]:
import plotly.graph_objs as go

fig = go.Figure()

fig.add_trace(pixels.heatmap_trace())
fig.add_trace(
    go.Scatter(x = gauss_xx, y = gauss_xy, yaxis = 'y2')
)
fig.add_trace(
    go.Scatter(y = gauss_yx, x = gauss_yy, xaxis = 'x2')
)

# Subplots layout set manually, adapted from
# https://plotly.com/python/2d-histogram-contour/#2d-histogram-contour-subplot
fig.update_layout(
    autosize = False,
    xaxis = dict(
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    yaxis = dict(
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    xaxis2 = dict(
        zeroline = False,
        domain = [0.85,1],
        showgrid = False
    ),
    yaxis2 = dict(
        zeroline = False,
        domain = [0.85,1],
        showgrid = False
    ),
    height = 600,
    width = 600,
    bargap = 0,
    hovermode = 'closest',
    showlegend = False
)

fig.show()

# 3. Complete Line Density Method Code


In [8]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 80_000,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.0000e+00, 0.0000e+00, 1.6870e+02, 6.0000e+02, 1.4280e+02],
       [1.0000e-01, 0.0000e+00, 1.6760e+02, 6.0000e+02, 3.1390e+02],
       [1.0000e-01, 0.0000e+00, 4.1000e+02, 6.0000e+02, 2.4010e+02],
       ...,
       [2.6443e+03, 0.0000e+00, 2.2480e+02, 6.0000e+02, 3.4280e+02],
       [2.6444e+03, 0.0000e+00, 1.3690e+02, 6.0000e+02, 1.4750e+02],
       [2.6444e+03, 0.0000e+00, 5.4220e+02, 6.0000e+02, 5.5280e+02]])

In [9]:
# Use the G-Means algorithm to track moving tracer
from scipy.optimize import curve_fit

# User-definable G-Means settings
fraction = 0.5              # Set all pixels lower than `fraction` of maximum pixel value to zero
resolution = (100, 100)     # Number of pixels in each dimension
sample_size = 100           # Number of LoRs in a sample

sample_start = 0
positions = []

# Define scaled Gauss model function to fit to data
def gauss(x, mu, sigma, scale):
    return scale / sigma / np.sqrt(2 * np.pi) * np.exp(-0.5 * (x - mu)**2 / sigma**2)

while sample_start + sample_size < len(lors):
    sample = lors[sample_start:sample_start + sample_size]
    pixels = pept.Pixels.from_lines(sample, resolution, verbose = False)

    # Find indices of maximum `pixels` value
    max_indices = np.unravel_index(pixels.argmax(), pixels.shape)

    # Fit the Gaussian curve to the pixel values in the horizontal and vertical pixel lines passing
    # through the maximum pixel value
    pixels_x = np.linspace(pixels.xlim[0] + pixels.pixel_size[0] / 2,
                        pixels.xlim[1] - pixels.pixel_size[0] / 2,
                        pixels.shape[0])
    pixels_y = np.linspace(pixels.ylim[0] + pixels.pixel_size[1] / 2,
                        pixels.ylim[1] - pixels.pixel_size[1] / 2,
                        pixels.shape[1])

    # Initial guesses
    p0_x = np.array([pixels_x[max_indices[0]], pixels.pixel_size[0], 1.])
    p0_y = np.array([pixels_y[max_indices[1]], pixels.pixel_size[1], 1.])

    opt_x, _ = curve_fit(gauss, pixels_x, pixels[:, max_indices[1]], p0_x, maxfev = 1000)
    opt_y, _ = curve_fit(gauss, pixels_y, pixels[max_indices[0], :], p0_y, maxfev = 1000)

    # Fitted Gauss curves around the X and Y pixel lines
    gauss_xx = np.linspace(pixels.xlim[0], pixels.xlim[1], 1000)
    gauss_xy = gauss(gauss_xx, *opt_x)

    gauss_yx = np.linspace(pixels.ylim[0], pixels.ylim[1], 1000)
    gauss_yy = gauss(gauss_yx, *opt_y)

    # The tracer's location is given by the Gaussians' centres
    location = np.array([opt_x[0], opt_y[0]])

    # Insert the tracer positions' timestamp at column 0
    location = np.insert(location, 0, lors[:, 0].mean())
    positions.append(location)

    sample_start += sample_size

positions = np.array(positions)
positions

/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning:

Covariance of the parameters could not be estimated



array([[1347.1026175 ,  317.08983931,  242.97475198],
       [1347.1026175 ,  314.5152207 ,  243.07195333],
       [1347.1026175 ,  316.99866626,  243.04927484],
       ...,
       [1347.1026175 ,  299.73945953,  291.07626357],
       [1347.1026175 ,  297.11627364,  291.13066613],
       [1347.1026175 ,  307.88349358,  290.90401381]])

In [10]:
from pept.visualisation import PlotlyGrapher2D

grapher = PlotlyGrapher2D(
    cols = 3,
    subplot_titles = ["First 400 LoRs", "First Pixellised Sample", "Positions Found"],
)

grapher.add_lines(lors[:400])
grapher.add_pixels(pixels, col = 2)
grapher.add_points(positions, col = 3)

grapher.show()